## Simulation

In [12]:
import numpy as np

def generate_markov_chain_sequence(initial_state, transition_matrix, steps):
    current_state = initial_state
    sequence = [current_state]

    for _ in range(steps - 1):
        current_state = np.random.choice(['normal', 'depressed', 'manic'], p=list(transition_matrix[current_state].values()))
        sequence.append(current_state)

    return sequence

def simulate_population(population_size, steps, transition_matrix):
    population = []
    for _ in range(population_size):
        initial_state = 'normal' 
        sequence = generate_markov_chain_sequence(initial_state, transition_matrix, steps)
        population.append(sequence)

    return population

def categorize_mental_health_state(markov_chain_output):
    consecutive_depression = 0
    has_mania = False
    has_two_consecutive_depression = False

    for state in markov_chain_output:
        if state == 'depressed':
            consecutive_depression += 1
            if consecutive_depression >= 2:
                has_two_consecutive_depression = True
        elif state == 'manic':
            has_mania = True
            consecutive_depression = 0  
        else:
            consecutive_depression = 0  

    if has_two_consecutive_depression and has_mania:
        return 'Mixed Bipolar'
    elif has_two_consecutive_depression:
        return 'MDD'
    elif has_mania:
        return 'Pure Mania'
    else:
        return 'Normal'

def analyze_population_distribution(population):
    distribution = {'Normal': 0, 'MDD': 0, 'Mixed Bipolar': 0, 'Pure Mania': 0}

    for sequence in population:
        category = categorize_mental_health_state(sequence)
        distribution[category] += 1

    return distribution

transition_matrix = {'normal': {'normal': 0.9947635315033879, 'depressed': 0.005236468496612034, 'manic': 0.0}, 'depressed': {'normal': 0.4961168967332019, 'depressed': 0.44711882565984223, 'manic': 0.05676427760695586}, 'manic': {'normal': 0.3144127430726594, 'depressed': 0.41431162105580444, 'manic': 0.27127563587153614}}


population_size = 100000
sequence_length = 52  # 52 weeks
num_simulations = 30

sum_percentages = {'Normal': 0, 'MDD': 0, 'Mixed Bipolar': 0, 'Pure Mania': 0}

for _ in range(num_simulations):
    simulated_population = simulate_population(population_size, sequence_length, transition_matrix)
    population_distribution = analyze_population_distribution(simulated_population)
    population_percentages = {k: (v / population_size) * 100 for k, v in population_distribution.items()}
    for category in sum_percentages:
        sum_percentages[category] += population_percentages[category]
average_percentages = {k: v / num_simulations for k, v in sum_percentages.items()}

average_percentages

{'Normal': 87.63683333333333,
 'MDD': 9.763700000000002,
 'Mixed Bipolar': 1.6119999999999997,
 'Pure Mania': 0.9874666666666668}

## DEAP

In [1]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

def generate_markov_chain_sequence(initial_state, transition_matrix, steps):
    current_state = initial_state
    sequence = [current_state]
    for _ in range(steps - 1):
        current_state = np.random.choice(['normal', 'depressed', 'manic'], p=list(transition_matrix[current_state].values()))
        sequence.append(current_state)
    return sequence

def simulate_population(population_size, steps, transition_matrix):
    population = []
    for _ in range(population_size):
        initial_state = 'normal'
        sequence = generate_markov_chain_sequence(initial_state, transition_matrix, steps)
        population.append(sequence)
    return population

def categorize_mental_health_state(markov_chain_output):
    consecutive_depression = 0
    has_mania = False
    has_two_consecutive_depression = False
    for state in markov_chain_output:
        if state == 'depressed':
            consecutive_depression += 1
            if consecutive_depression >= 2:
                has_two_consecutive_depression = True
        elif state == 'manic':
            has_mania = True
            consecutive_depression = 0  
        else:
            consecutive_depression = 0  
    if has_two_consecutive_depression and has_mania:
        return 'Mixed Bipolar'
    elif has_two_consecutive_depression:
        return 'MDD'
    elif has_mania:
        return 'Pure Mania'
    else:
        return 'Normal'

def analyze_population_distribution(population):
    distribution = {'Normal': 0, 'MDD': 0, 'Mixed Bipolar': 0, 'Pure Mania': 0}
    for sequence in population:
        category = categorize_mental_health_state(sequence)
        distribution[category] += 1
    return distribution


target_distribution = {'Normal': 87.5, 'MDD': 10, 'Mixed Bipolar': 2, 'Pure Mania': 0.5}

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def generate_individual():
    return [random.random() for _ in range(9)]

toolbox.register("individual", tools.initIterate, creator.Individual, generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def normalize_probabilities(individual):
    for i in range(0, len(individual), 3):
        total = sum(max(0, p) for p in individual[i:i+3])  
        if total > 0:
            for j in range(3):
                individual[i+j] = max(0, individual[i+j]) / total
        else:
            individual[i:i+3] = [1/3, 1/3, 1/3]  
    return individual

def mutate(individual):
    for i in range(len(individual)):
        if random.random() < 0.1:  
            individual[i] += random.gauss(0, 0.1) 
            individual[i] = max(0, individual[i])  
    return individual,

toolbox.register("mutate", mutate)
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("select", tools.selTournament, tournsize=3)


def evaluate(individual):
    individual = normalize_probabilities(individual)
    transition_matrix = {
        'normal': {'normal': individual[0], 'depressed': individual[1], 'manic': individual[2]},
        'depressed': {'normal': individual[3], 'depressed': individual[4], 'manic': individual[5]},
        'manic': {'normal': individual[6], 'depressed': individual[7], 'manic': individual[8]}
    }
    simulated_population = simulate_population(1000, 52, transition_matrix)
    population_distribution = analyze_population_distribution(simulated_population)
    population_percentages = {k: (v / 1000) * 100 for k, v in population_distribution.items()}
    fitness = -sum(abs(population_percentages[k] - target_distribution[k]) for k in target_distribution)
    return fitness,

toolbox.register("evaluate", evaluate)


population_size = 300
num_generations = 50

population = toolbox.population(n=population_size)

for gen in range(num_generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))


best_ind = tools.selBest(population, 1)[0]
best_transition_matrix = {
    'normal': {'normal': best_ind[0], 'depressed': best_ind[1], 'manic': best_ind[2]},
    'depressed': {'normal': best_ind[3], 'depressed': best_ind[4], 'manic': best_ind[5]},
    'manic': {'normal': best_ind[6], 'depressed': best_ind[7], 'manic': best_ind[8]}
}

print("Best Transition Matrix:", best_transition_matrix)


Best Transition Matrix: {'normal': {'normal': 0.9947635315033879, 'depressed': 0.005236468496612034, 'manic': 0.0}, 'depressed': {'normal': 0.4961168967332019, 'depressed': 0.44711882565984223, 'manic': 0.05676427760695586}, 'manic': {'normal': 0.3144127430726594, 'depressed': 0.41431162105580444, 'manic': 0.27127563587153614}}


# Beta Distribution

In [126]:
import numpy as np

def generate_markov_chain_sequence(initial_state, transition_matrix, steps):
    current_state = initial_state
    sequence = [current_state]

    for _ in range(steps - 1):
        current_state = np.random.choice(['normal', 'depressed', 'manic'], p=list(transition_matrix[current_state].values()))
        sequence.append(current_state)

    return sequence
population = []
def simulate_population(population_size, steps, transition_matrix):
    
    for _ in range(population_size):
        initial_state = 'normal' 
        sequence = generate_markov_chain_sequence(initial_state, transition_matrix, steps)
        population.append(sequence)

    return population

def categorize_mental_health_state(markov_chain_output):
    consecutive_depression = 0
    has_mania = False
    has_two_consecutive_depression = False

    for state in markov_chain_output:
        if state == 'depressed':
            consecutive_depression += 1
            if consecutive_depression >= 2:
                has_two_consecutive_depression = True
        elif state == 'manic':
            has_mania = True
            consecutive_depression = 0  
        else:
            consecutive_depression = 0  

    if has_two_consecutive_depression and has_mania:
        return 'Mixed Bipolar'
    elif has_two_consecutive_depression:
        return 'MDD'
    elif has_mania:
        return 'Pure Mania'
    else:
        return 'Normal'

def analyze_population_distribution(population):
    distribution = {'Normal': 0, 'MDD': 0, 'Mixed Bipolar': 0, 'Pure Mania': 0}

    for sequence in population:
        category = categorize_mental_health_state(sequence)
        distribution[category] += 1

    return distribution


ftransition_matrix = {'normal': {'normal': 0.9947635315033879, 'depressed': 0.005236468496612034, 'manic': 0.0}, 'depressed': {'normal': 0.4961168967332019, 'depressed': 0.44711882565984223, 'manic': 0.05676427760695586}, 'manic': {'normal': 0.3144127430726594, 'depressed': 0.41431162105580444, 'manic': 0.27127563587153614}}
def mean_variance_to_alpha_beta(mean, variance):
    """Convert mean and variance of a Beta distribution to alpha and beta parameters."""
    alpha = ((1 - mean) / variance - 1 / mean) * mean**2
    beta = alpha * (1 / mean - 1)
    return alpha, beta

def normalize_transition_matrix(transition_matrix):
    """
    Normalize the rows of a transition matrix so that the values in each row add up to 1.

    Args:
    - transition_matrix (dict): A dictionary of dictionaries representing the transition matrix.

    Returns:
    - dict: A normalized transition matrix.
    """
    normalized_matrix = {}
    
    for state, transitions in transition_matrix.items():
        # Calculate the sum of the row values
        row_sum = sum(transitions.values())
        
        # Normalize each value in the row if row_sum is not zero
        if row_sum > 0:
            normalized_matrix[state] = {next_state: value / row_sum for next_state, value in transitions.items()}
        else:
            # Handle the case where row_sum is 0 to avoid division by zero
            # This could involve setting the transitions equally or some other logic
            num_transitions = len(transitions)
            normalized_matrix[state] = {next_state: 1 / num_transitions for next_state in transitions}
    
    return normalized_matrix

def new_value(old_value):
    variance = 1e-10
    if old_value == 0:
        old_value = 1e-6
    alpha, beta = mean_variance_to_alpha_beta(old_value, variance)
    return np.random.beta(alpha, beta)  # Example adjustment

# Iterate over the ftransition_matrix and update its values
for state in ftransition_matrix:
    for next_state in ftransition_matrix[state]:
        # Update the value with a new number
        old_value = ftransition_matrix[state][next_state]
        ftransition_matrix[state][next_state] = new_value(old_value)

transition_matrix = normalize_transition_matrix(ftransition_matrix)
print(transition_matrix)




population_size = 100000
sequence_length = 52  # 52 weeks
num_simulations = 1

sum_percentages = {'Normal': 0, 'MDD': 0, 'Mixed Bipolar': 0, 'Pure Mania': 0}

for _ in range(num_simulations):
    for i in range(population_size):
        simulated_population = simulate_population(1, sequence_length, transition_matrix)
    population_distribution = analyze_population_distribution(simulated_population)
    population_percentages = {k: (v / population_size) * 100 for k, v in population_distribution.items()}
    for category in sum_percentages:
        sum_percentages[category] += population_percentages[category]
average_percentages = {k: v / num_simulations for k, v in sum_percentages.items()}

average_percentages

{'normal': {'normal': 0.9947552154697274, 'depressed': 0.005244784530269323, 'manic': 3.1549462876539726e-15}, 'depressed': {'normal': 0.49611964750398774, 'depressed': 0.44711461479397935, 'manic': 0.05676573770203284}, 'manic': {'normal': 0.31441540592541145, 'depressed': 0.4143201461538295, 'manic': 0.27126444792075904}}


In [85]:
import numpy as np

def mean_variance_to_alpha_beta(mean, variance=1e-6):
    """Convert mean and variance of a Beta distribution to alpha and beta parameters."""
    # Adjust mean values very close to 0 or 1 to avoid division by zero or negative alpha/bet
    # Calculate alpha and beta using the adjusted mean
    alpha = mean * (mean * (1 - mean) / variance - 1)
    beta = (1 - mean) * (mean * (1 - mean) / variance - 1)
    if mean == 0:
        alpha , beta = 0
    return alpha, beta

def normalize_transition_matrix(transition_matrix):
    normalized_matrix = {}
    for state, transitions in transition_matrix.items():
        row_sum = sum(transitions.values())
        normalized_matrix[state] = {next_state: value / row_sum for next_state, value in transitions.items()}
    return normalized_matrix

def generate_and_normalize_transition_matrix(base_transition_matrix, variance=1e-6):
    individual_matrix = {}
    for state in base_transition_matrix:
        individual_matrix[state] = {}
        for next_state in base_transition_matrix[state]:
            old_value = base_transition_matrix[state][next_state] if base_transition_matrix[state][next_state] > 0 else 1e-9
            alpha, beta = mean_variance_to_alpha_beta(old_value, variance)
            # Ensure alpha and beta are positive
            
            
            if alpha > 0:
                new_value = np.random.beta(alpha, beta)
            else:
                new_value = 0
            individual_matrix[state][next_state] = new_value
    return normalize_transition_matrix(individual_matrix)

def generate_markov_chain_sequence(initial_state, transition_matrix, steps):
    current_state = initial_state
    sequence = [current_state]
    for _ in range(steps - 1):
        current_state = np.random.choice(['normal', 'depressed', 'manic'], p=list(transition_matrix[current_state].values()))
        sequence.append(current_state)
    return sequence

def simulate_population(population_size, steps, base_transition_matrix):
    population = []
    for _ in range(population_size):
        initial_state = 'normal'
        individual_matrix = generate_and_normalize_transition_matrix(base_transition_matrix)
        sequence = generate_markov_chain_sequence(initial_state, individual_matrix, steps)
        population.append(sequence)
    return population


def categorize_mental_health_state(markov_chain_output):
    """Categorize the mental health state of an individual based on their Markov chain sequence."""
    consecutive_depression = 0
    has_mania = False
    has_two_consecutive_depression = False
    for state in markov_chain_output:
        if state == 'depressed':
            consecutive_depression += 1
            if consecutive_depression >= 2:
                has_two_consecutive_depression = True
        elif state == 'manic':
            has_mania = True
            consecutive_depression = 0
        else:
            consecutive_depression = 0
    if has_two_consecutive_depression and has_mania:
        return 'Mixed Bipolar'
    elif has_two_consecutive_depression:
        return 'MDD'
    elif has_mania:
        return 'Pure Mania'
    else:
        return 'Normal'

def analyze_population_distribution(population):
    """Analyze the distribution of mental health categories in the simulated population."""
    distribution = {'Normal': 0, 'MDD': 0, 'Mixed Bipolar': 0, 'Pure Mania': 0}
    for sequence in population:
        category = categorize_mental_health_state(sequence)
        distribution[category] += 1
    return distribution

# Base transition matrix
base_transition_matrix = {
    'normal': {'normal': 0.9947635315033879, 'depressed': 0.005236468496612034, 'manic': 0.0},
    'depressed': {'normal': 0.4961168967332019, 'depressed': 0.44711882565984223, 'manic': 0.05676427760695586},
    'manic': {'normal': 0.3144127430726594, 'depressed': 0.41431162105580444, 'manic': 0.27127563587153614}
}

# Simulation parameters
population_size = 100000
sequence_length = 52  # For example, 52 weeks

# Simulate the population
simulated_population = simulate_population(population_size, sequence_length, base_transition_matrix)

# Analyze the population distribution
population_distribution = analyze_population_distribution(simulated_population)

# Display the results
print(population_distribution)


{'Normal': 87615, 'MDD': 9828, 'Mixed Bipolar': 1585, 'Pure Mania': 972}


In [86]:
total_population = sum(population_distribution.values())

# Calculate the percentage for each state
percentage_distribution = {state: (count / total_population) * 100 for state, count in population_distribution.items()}

percentage_distribution

{'Normal': 87.615,
 'MDD': 9.828000000000001,
 'Mixed Bipolar': 1.585,
 'Pure Mania': 0.972}